In [1]:
import numpy as np
import pandas as pd
import math
import statistics
import matplotlib.pyplot as plt
from transformers import pipeline
from sklearn.model_selection import train_test_split, cross_val_score,KFold, cross_val_predict, GridSearchCV,RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.datasets import make_classification
from sklearn.neural_network import MLPClassifier


In [2]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

## Download All Policies - Train Set

In [3]:
##Privacy Policies download:
websites = ['Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson']

sequence_to_classify = []

for i in websites:
    try:
        with open((i+".txt"), "r") as f:
            i = f.readlines()
            sequence_to_classify.append(i)
    except:
        with open((i+".txt"), "r", encoding='cp1252') as f:
            i = f.readlines()
            sequence_to_classify.append(i)

### Train Set Vectors:

In [4]:
third_party_train = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1]


### Third Party Transfer

In [5]:
##Third Party Transfer Matrix:

Phrases = ['Privacy Policy'
           ,'we share personal information'
           ,'the data is being transferred to third party'
           ,'We will share your personal data with third parties'
           ,'we will transferred your data to third party'
           ,'third parties have access to your Personal Data'
           ,'We will share your personal data with third parties'
           ,'Guest User personal data may be shared with third parties for these purposes'
           ,'we do not share your personal data with third parties outside of the organization unless one of the following circumstances applies'
           ,'we may also transfer such personal data to third parties'
           ,'We will not sell or lease your personal information to third parties unless we have your permission or are required by law to do so']

##Matrix creation:
matrix_third_party_ML = pd.DataFrame(columns = Phrases)
matrix_third_party_ML['Privacy Policy'] = websites
matrix_third_party_ML.set_index('Privacy Policy', inplace=True)

##Filling the matrix:
line = 0
for j in sequence_to_classify:
    for i in matrix_third_party_ML:
        matrix_third_party_ML.loc[websites[line],i] = classifier(str(j), str(i))["scores"][0]
    line += 1
    
matrix_third_party_ML


,we share personal information,the data is being transferred to third party,We will share your personal data with third parties,we will transferred your data to third party,third parties have access to your Personal Data,We will share your personal data with third parties,Guest User personal data may be shared with third parties for these purposes,we do not share your personal data with third parties outside of the organization unless one of the following circumstances applies,we may also transfer such personal data to third parties,We will not sell or lease your personal information to third parties unless we have your permission or are required by law to do so
Privacy Policy,,,,,,,,,,
Google,0.988903,0.96316,0.946369,0.925576,0.923332,0.946369,0.983469,0.820747,0.991814,0.487503
Aws,0.957286,0.917251,0.926605,0.844959,0.920556,0.926605,0.943358,0.758988,0.955437,0.479819
AliExpress,0.804805,0.835749,0.881253,0.80256,0.863816,0.881253,0.89855,0.666077,0.977357,0.301057
Meta,0.986078,0.947642,0.960855,0.920288,0.945334,0.960855,0.978357,0.956771,0.988766,0.731819
TikTok,0.910168,0.881434,0.865846,0.824552,0.878973,0.865846,0.939493,0.858876,0.925442,0.445399
YouTube,0.984711,0.943645,0.936913,0.885544,0.941345,0.936913,0.977639,0.765031,0.992065,0.376592
Waze,0.944814,0.917379,0.951227,0.924603,0.876499,0.951227,0.963058,0.750747,0.989674,0.526696
Wix,0.979383,0.93116,0.939625,0.833984,0.931682,0.939625,0.987172,0.514952,0.98372,0.12436
Bookings,0.992651,0.978623,0.952655,0.967409,0.977454,0.952655,0.990119,0.86005,0.995551,0.50852


### Test The Model - ML

In [6]:
# X_train, X_test, y_train, y_test = train_test_split(matrix_policy_change, policy_change_train
#                                                     , test_size=0.5, random_state=42)

# Manual division so we will have all options (1/0) at both groups
X_train = matrix_third_party_ML.iloc[:35,:] 
X_test = matrix_third_party_ML.iloc[34:,:]
y_train = third_party_train[:35]
y_test = third_party_train[34:]

##Decission Tree Classifaier
regressor_tree = DecisionTreeClassifier()
regressor_tree = regressor_tree.fit(X_train, y_train)
pred_tree = regressor_tree.predict(X_test)
print('Decission Tree Classifaier:', pred_tree)
print(classification_report(y_test, pred_tree))

##Random Forest Classifaier
regressor_forest = RandomForestClassifier()
regressor_forest = regressor_forest.fit(X_train, y_train)
pred_forest = regressor_forest.predict(X_test).round()
print('Random Forest Classifaier:', pred_forest)
print(classification_report(y_test, pred_forest))

##Logistic Reg Classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)
pred_clf = clf.predict(X_test)
print('Logistic Reg Classifier:', pred_clf)
print(classification_report(y_test, pred_clf))

##SVM
SVM = SVC(kernel='linear')
SVM = SVM.fit(X_train, y_train)
pred_SVM = SVM.predict(X_test)
print('SVM:', pred_SVM)
print(classification_report(y_test, pred_SVM))

##KNN
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
KNN = KNN.fit(X_train, y_train)
pred_KNN = KNN.predict(X_test)
print('KNN:', pred_KNN)
print(classification_report(y_test, pred_KNN))

##neural_network
neural_network = MLPClassifier(random_state=1, max_iter=300)
neural_network = neural_network.fit(X_train, y_train)
pred_neural_network = neural_network.predict(X_test)
print('neural_network:', pred_neural_network)
print(classification_report(y_test, pred_neural_network))

Decission Tree Classifaier: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.87      1.00      0.93        13

    accuracy                           0.87        15
   macro avg       0.43      0.50      0.46        15
weighted avg       0.75      0.87      0.80        15

Random Forest Classifaier: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.87      1.00      0.93        13

    accuracy                           0.87        15
   macro avg       0.43      0.50      0.46        15
weighted avg       0.75      0.87      0.80        15

Logistic Reg Classifier: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.87      1.00      0.93        13

    ac

/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalb

neural_network: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.87      1.00      0.93        13

    accuracy                           0.87        15
   macro avg       0.43      0.50      0.46        15
weighted avg       0.75      0.87      0.80        15



/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
##Third Party Transfer Matrix:

##Matrix creation:
matrix_third_party_Comparison = pd.DataFrame(columns = ['Phrase',
                                               'Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson'])
matrix_third_party_Comparison['Phrase'] = ['Third Party Transfer']

##Filling the matrix:
websites_index = 0
line = 0
pred = 0
third_party_train_pred_Comparison = []
c = ['your data can transfer to third party','your data is not being transfered to third party']

for j in sequence_to_classify:
    a = classifier(str(j),c)
    if a["labels"][0]==c[0] :#and a["scores"][0]>0.6:
        pred = 1
    else:
        pred = 0  
    matrix_third_party_Comparison[websites[websites_index]].loc[line] = pred
    third_party_train_pred_Comparison.append(pred)
    websites_index += 1
    pred = 0

third_party_train_pred_Comparison

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [12]:
index = 0
TP = 0
FP = 0
FN = 0
TN = 0
for i in third_party_train:
    if third_party_train_pred_Comparison[index]==1 and third_party_train[index]==1:
        TP +=1
    elif third_party_train_pred_Comparison[index]==0 and third_party_train[index]==0:
        TN +=1
    elif third_party_train_pred_Comparison[index]==0 and third_party_train[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1

recall = TP/(TP+FN)
specificity = TN/(TN+FP)
Precision = TP/(TP+FP)
Accuracy = (TP+TN)/(TP+FP+FN+TN) 
F1 = (TP)/(TP+(0.5*(FP+FN)))

print("recall:" , recall)
print("specificity:" , specificity)
print("Precision:" , Precision)
print("Accuracy:" , Accuracy)
print("F1 Score:" , F1)

recall: 1.0
specificity: 0.0
Precision: 0.9387755102040817
Accuracy: 0.9387755102040817
F1 Score: 0.968421052631579


In [11]:
##Privacy Policy Change Matrix:
matrix_third_party_Median = matrix_third_party_ML.transpose()

matrix_third_party_Median.loc['Median'] =  matrix_third_party_Median.median()
matrix_third_party_Median.loc['Median']['Privacy Policy'] = 'Median'

third_party_train_pred_Median = []
websites_index = 0

for i in matrix_third_party_Median:
    num = matrix_third_party_Median[websites[websites_index]].loc['Median']
    if num >0.9:
        third_party_train_pred_Median.append(1)
    else:
        third_party_train_pred_Median.append(0)
    websites_index += 1
    
third_party_train_pred_Median

[1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1]

In [12]:
index = 0
TP = 0
FP = 0
FN = 0
TN = 0

for i in third_party_train:
    if third_party_train_pred_Median[index]==1 and third_party_train[index]==1:
        TP +=1
    elif third_party_train_pred_Median[index]==0 and third_party_train[index]==0:
        TN +=1
    elif third_party_train_pred_Median[index]==0 and third_party_train[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1

recall = TP/(TP+FN)
Precision = TP/(TP+FP)
Accuracy = (TP+TN)/(TP+FP+FN+TN) 
F1 = (TP)/(TP+(0.5*(FP+FN)))

print("recall:" , recall)
print("Precision:" , Precision)
print("Accuracy:" , Accuracy)
print("F1 Score:" , F1)

recall: 0.8043478260869565
Precision: 0.9487179487179487
Accuracy: 0.7755102040816326
F1 Score: 0.8705882352941177
